In [4]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import local.lib.mlutils
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
%matplotlib inline

/content/local/lib/mlutils.py:445: SyntaxWarning: invalid escape sequence '\m'
  time_str = "fit time  (TRAIN)    %.2f $\mu$secs"%(fitting_time*1000)


Cargo el train.csv

In [6]:
d = pd.read_csv("train.csv")
print (d.shape)

(692500, 21)


Primer Head para ver los datos del Kaggle

In [7]:
d.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


Como en el head no se ven bien todas las columnas las mirare como lista

In [11]:
d.columns
list(d.columns)


['ID',
 'PERIODO_ACADEMICO',
 'E_PRGM_ACADEMICO',
 'E_PRGM_DEPARTAMENTO',
 'E_VALORMATRICULAUNIVERSIDAD',
 'E_HORASSEMANATRABAJA',
 'F_ESTRATOVIVIENDA',
 'F_TIENEINTERNET',
 'F_EDUCACIONPADRE',
 'F_TIENELAVADORA',
 'F_TIENEAUTOMOVIL',
 'E_PRIVADO_LIBERTAD',
 'E_PAGOMATRICULAPROPIO',
 'F_TIENECOMPUTADOR',
 'F_TIENEINTERNET.1',
 'F_EDUCACIONMADRE',
 'RENDIMIENTO_GLOBAL',
 'INDICADOR_1',
 'INDICADOR_2',
 'INDICADOR_3',
 'INDICADOR_4']

Aqui miramos cuantos registros hay por periodo para saber con que cantidad de datos estare trabajando

In [25]:
d['PERIODO_ACADEMICO'].value_counts()


,count
PERIODO_ACADEMICO,
20195,180873
20203,171838
20212,171412
20183,164818
20194,1472
20213,1178
20202,490
20184,254
20196,165


Aqui ya empiezo a experimentar con los datos, queria ver cuantos registros de programas academicos teniamos para trabajar, pero tenemos almenos 958 programas academicos, por lo menos pudimos ver cuales son los programas que mas se repiten con 53244 cursos de derecho en 9 semestres...


In [31]:
d['E_PRGM_ACADEMICO'].value_counts()

,count
E_PRGM_ACADEMICO,
DERECHO,53244
ADMINISTRACION DE EMPRESAS,51902
CONTADURIA PUBLICA,39664
PSICOLOGIA,31932
INGENIERIA INDUSTRIAL,28481
...,...
DEPORTE Y ACTIVIDADA FISICA,1
QUIMICA Y FARMACIA,1
"LICENCIATURA EN EDUCACON FISICA, RECREACION Y DEPORTES",1


In [33]:
program_counts = d['E_PRGM_ACADEMICO'].value_counts()
programs_greater_than_1 = program_counts[program_counts > 1000]
print(programs_greater_than_1)

E_PRGM_ACADEMICO
DERECHO                                                   53244
ADMINISTRACION DE EMPRESAS                                51902
CONTADURIA PUBLICA                                        39664
PSICOLOGIA                                                31932
INGENIERIA INDUSTRIAL                                     28481
                                                          ...  
ANTROPOLOGIA                                               1052
LICENCIATURA EN EDUCACIÓN FÍSICA, RECREACIÓN Y DEPORTE     1034
COMUNICACIÓN SOCIAL                                        1031
INGENIERIA INFORMATICA                                     1020
JURISPRUDENCIA                                             1011
Name: count, Length: 87, dtype: int64


Aunque hay excepciones podemos ver que en derecho almenos solo re recluta una vez al año en un semestre especifico, podriamos ver a los datos atipicos como estudiantes de intercambio o otra variable, ademas noto que puede que lo que antes note como veces que se repite el programa sea mas los alumnos que ingresan al programa

In [35]:
df_filtrado = d[d['E_PRGM_ACADEMICO'] == "DERECHO"]
df_filtrado.groupby(['E_PRGM_ACADEMICO', 'PERIODO_ACADEMICO']).size()


E_PRGM_ACADEMICO  PERIODO_ACADEMICO
DERECHO           20183                12486
                  20184                   13
                  20194                   22
                  20195                13368
                  20196                   10
                  20202                    6
                  20203                13940
                  20212                13364
                  20213                   35
dtype: int64

Decidi ver los datos de indicadores pero no les encuentro mucho sentido, puede ser una nota? no me queda claro

In [19]:
d.groupby(['INDICADOR_1']).size()
d['INDICADOR_3'].value_counts()


,count
INDICADOR_3,
0.276,10424
0.284,10359
0.288,10349
0.296,10337
0.279,10292
...,...
0.179,63
0.183,59
0.181,59


Otro filtro esta vez para revizar los datos de los programas academicos, donde el rendimiento fuera alto.

In [23]:
df_filtrado = d[d['PERIODO_ACADEMICO'] == 20212]
df_reducido = df_filtrado[['E_PRGM_ACADEMICO', 'RENDIMIENTO_GLOBAL']]
df_ordenado = df_reducido.sort_values(by='RENDIMIENTO_GLOBAL', ascending=True)
top_20 = df_ordenado.head(20)
top_20


,E_PRGM_ACADEMICO,RENDIMIENTO_GLOBAL
346993,INGENIERIA DE SISTEMAS,alto
455670,INGENIERIA INDUSTRIAL,alto
162956,ARCHIVISTICA,alto
455658,INGENIERIA DE SISTEMAS,alto
455648,COMUNICACIONES,alto
162971,INGENIERIA AMBIENTAL,alto
162976,ADMINISTRACION DE EMPRESAS,alto
455594,ADMINISTRACION DE EMPRESAS,alto
162991,LICENCIATURA EN EDUCACION BASICA CON ENFASIS E...,alto
455577,INGENIERIA DE SISTEMAS Y COMPUTACION,alto


In [36]:
for i in d.columns:
    print ("%20s"%i, np.sum(d[i].isna()))

                  ID 0
   PERIODO_ACADEMICO 0
    E_PRGM_ACADEMICO 0
 E_PRGM_DEPARTAMENTO 0
E_VALORMATRICULAUNIVERSIDAD 6287
E_HORASSEMANATRABAJA 30857
   F_ESTRATOVIVIENDA 32137
     F_TIENEINTERNET 26629
    F_EDUCACIONPADRE 23178
     F_TIENELAVADORA 39773
    F_TIENEAUTOMOVIL 43623
  E_PRIVADO_LIBERTAD 0
E_PAGOMATRICULAPROPIO 6498
   F_TIENECOMPUTADOR 38103
   F_TIENEINTERNET.1 26629
    F_EDUCACIONMADRE 23664
  RENDIMIENTO_GLOBAL 0
         INDICADOR_1 0
         INDICADOR_2 0
         INDICADOR_3 0
         INDICADOR_4 0
